# Deskripsi Proyek

membangun model yang dapat memprediksi credit risk menggunakan dataset yang disediakan oleh company yang terdiri dari data pinjaman yang diterima dan yang ditolak.

Datasets: loan_data_2007_2014

## Library

In [1]:
import pandas as pd, numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

## Pre-Processing

In [2]:
path = '/home/jovyan/work/IDx_Partner/loan_data_2007_2014.csv'

In [3]:
df = pd.read_csv(path,index_col = 0, low_memory=False)

In [4]:
df.sample()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
347037,21090558,23373258,30000,30000,30000.0,60 months,17.57,754.81,D,D4,...,NaN,NaN,NaN,NaN,NaN,NaN,62000.0,NaN,NaN,NaN


In [5]:
df.shape

(466285, 74)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466285 entries, 0 to 466284
Data columns (total 74 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           466285 non-null  int64  
 1   member_id                    466285 non-null  int64  
 2   loan_amnt                    466285 non-null  int64  
 3   funded_amnt                  466285 non-null  int64  
 4   funded_amnt_inv              466285 non-null  float64
 5   term                         466285 non-null  object 
 6   int_rate                     466285 non-null  float64
 7   installment                  466285 non-null  float64
 8   grade                        466285 non-null  object 
 9   sub_grade                    466285 non-null  object 
 10  emp_title                    438697 non-null  object 
 11  emp_length                   445277 non-null  object 
 12  home_ownership               466285 non-null  object 
 13 

In [7]:
df.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
count,4.662850e+05,4.662850e+05,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,4.662810e+05,466285.000000,466256.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,3.960090e+05,0.0,0.0,0.0
mean,1.307973e+07,1.459766e+07,14317.277577,14291.801044,14222.329888,13.829236,432.061201,7.327738e+04,17.218758,0.284678,...,NaN,NaN,NaN,NaN,NaN,NaN,3.037909e+04,NaN,NaN,NaN
std,1.089371e+07,1.168237e+07,8286.509164,8274.371300,8297.637788,4.357587,243.485550,5.496357e+04,7.851121,0.797365,...,NaN,NaN,NaN,NaN,NaN,NaN,3.724713e+04,NaN,NaN,NaN
min,5.473400e+04,7.047300e+04,500.000000,500.000000,0.000000,5.420000,15.670000,1.896000e+03,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN
25%,3.639987e+06,4.379705e+06,8000.000000,8000.000000,8000.000000,10.990000,256.690000,4.500000e+04,11.360000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,1.350000e+04,NaN,NaN,NaN
50%,1.010790e+07,1.194108e+07,12000.000000,12000.000000,12000.000000,13.660000,379.890000,6.300000e+04,16.870000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,2.280000e+04,NaN,NaN,NaN
75%,2.073121e+07,2.300154e+07,20000.000000,20000.000000,19950.000000,16.490000,566.580000,8.896000e+04,22.780000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,3.790000e+04,NaN,NaN,NaN
max,3.809811e+07,4.086083e+07,35000.000000,35000.000000,35000.000000,26.060000,1409.990000,7.500000e+06,39.990000,29.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,9.999999e+06,NaN,NaN,NaN


### Informasi setiap Fitur

In [8]:
#Melakukan pengecekan value setiap fitur
#yang dicantumkan di notebook ini sebagai contoh saya ambil dari purpose dan grade
df['purpose'].value_counts()

debt_consolidation    274195
credit_card           104157
home_improvement       26537
other                  23690
major_purchase          9828
small_business          7013
car                     5397
medical                 4602
moving                  2994
vacation                2487
wedding                 2343
house                   2269
educational              422
renewable_energy         351
Name: purpose, dtype: int64

In [9]:
df['grade'].value_counts()

B    136929
C    125293
D     76888
A     74867
E     35757
F     13229
G      3322
Name: grade, dtype: int64

Selain dari 2 sample di atas, dilakukan juga pengecekan terhadap nilai seluruh fitur yang ada dan didapatkan kesimpulan untuk membagi dataset menjadi 2 kelompok, yaitu:
1. Dataset yang berfungsi untuk memprediksi Credit Risk yang nantinya akan dilakukan machine learning Supervised
2. Dataset yang berfungsi untuk mengelompokan dan melabeli Credit risk yang nantinya akan dilakukan machine learning Unsupervised

dari ke-2 dataset tersebut akan ada kolom **id** sebagai penanda, dan **id** disini akan dijadikan index untuk menghubungkan ke-2 dataset.

#### Dataset supervised

In [10]:
supervised_col = ['id','loan_amnt','term', 'int_rate', 'installment', 'grade', 'sub_grade',
       'emp_length', 'home_ownership', 'annual_inc','loan_status','purpose','addr_state','dti']

In [11]:
#Membuat dataframe baru yang fiturnya akan berfokus untuk memprediksi credit risk
#dengan index nya berdasarkan id
df_supervised = df[supervised_col].set_index('id')
df_supervised.sample()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,loan_status,purpose,addr_state,dti
id,,,,,,,,,,,,,
7069293,24375,60 months,25.83,727.35,G,G2,NaN,MORTGAGE,55224.0,Current,debt_consolidation,SC,33.55


#### Dataset Unsupervised

In [12]:
unsupervised_col = ['id','dti','delinq_2yrs','inq_last_6mths','open_acc', 
                    'pub_rec', 'revol_bal','initial_list_status','collection_recovery_fee',
                   'acc_now_delinq']

In [13]:
#Membuat dataframe baru yang fitur nya akan berfokus untuk memngelompokan credit risk 
#dengan index nya berdasarkan id
df_unsupervised = df[unsupervised_col].set_index('id')
df_unsupervised.sample()

,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,initial_list_status,collection_recovery_fee,acc_now_delinq
id,,,,,,,,,
26298369,17.91,0.0,0.0,5.0,0.0,1767,w,0.0,0.0


### Memperbaiki nilai

#### Dataset Supervised

fitur df_supervised
1. loan_amnt = jumlah peminjaman 
2. term = lama nya peminjaman (36 bulan dan 60 bulan)
3. int_rate = suku bunga pinjaman
4. installment = pembayaran bulanan
5. grade = LC menilai pinjaman (A - G)
6. subgrade = subgrade dari pinjaman (_1 - _5)
7. empt_length = lama bekerja (0-10+)
8. home_ownership =status kepemilikan rumah
9. annual_inc = pendapatan tahunan
10. loan_status = status pinjaman saat ini
9. purpose = kepentingan pinjaman
10. addr_state = negara dari peminjam
11. dti = Rasio dari hutang bulanan terhadap total utang dibagi dengan pendapatan bulanan

In [14]:
df_supervised.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466285 entries, 1077501 to 9199665
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   loan_amnt       466285 non-null  int64  
 1   term            466285 non-null  object 
 2   int_rate        466285 non-null  float64
 3   installment     466285 non-null  float64
 4   grade           466285 non-null  object 
 5   sub_grade       466285 non-null  object 
 6   emp_length      445277 non-null  object 
 7   home_ownership  466285 non-null  object 
 8   annual_inc      466281 non-null  float64
 9   loan_status     466285 non-null  object 
 10  purpose         466285 non-null  object 
 11  addr_state      466285 non-null  object 
 12  dti             466285 non-null  float64
dtypes: float64(4), int64(1), object(8)
memory usage: 49.8+ MB


##### emp_length

In [15]:
#cek missing value dari emp_length
df_supervised['emp_length'].unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

In [16]:
#mengubah nilai nan dan <1 year menjadi 0 sesuai ketentuan LCDataDictionary RejectStats
df_supervised.loc[(df_supervised['emp_length'].isna()) | (df_supervised['emp_length'] == '< 1 year'),'emp_length']= 0

In [17]:
#mengubah tipe data emp__length dari string menjadi integer
df_supervised['emp_length'] = df_supervised['emp_length'].apply(lambda x: re.findall('([0-9]+)',str(x))[0]).astype(int)

In [18]:
df_supervised['emp_length'].value_counts()

10    150049
0      57273
2      41373
3      36596
5      30774
1      29622
4      28023
7      26180
6      26112
8      22395
9      17888
Name: emp_length, dtype: int64

emp_length adalah Panjang pekerjaan dalam beberapa tahun. Nilai yang mungkin adalah antara 0 sampai 10 di mana 0 berarti kurang dari satu tahun dan 10 berarti sepuluh tahun atau lebih.

##### anual_inc

In [19]:
#Cek jumlah nilai yang hilang
df_supervised['annual_inc'].isna().sum()

4

In [20]:
#memasukan index yang akan di hapus
drop_index = df_supervised[df_supervised['annual_inc'].isna()].index.values

In [21]:
#dilakukan drop berdasarkan index kepada 2 dataset supervised dan unsupervised
df_supervised = df_supervised.drop(drop_index)
df_unsupervised = df_unsupervised.drop(drop_index)

Dikarenakan jumlah missing value dari annual_inc relatif kecil dan penghapusan ini tidak mengurangi representativitas dataset secara signifikan.

##### home_ownership

In [22]:
#unique value dari ownership
df_supervised['home_ownership'].unique()

array(['RENT', 'OWN', 'MORTGAGE', 'OTHER', 'NONE', 'ANY'], dtype=object)

In [23]:
df_supervised['home_ownership'].value_counts()

MORTGAGE    235875
RENT        188473
OWN          41704
OTHER          182
NONE            46
ANY              1
Name: home_ownership, dtype: int64

In [24]:
#merubah nilai None dan ANY menjadi OTHER
df_supervised.loc[(df_supervised['home_ownership'] == 'NONE') | 
                  (df_supervised['home_ownership'] == 'ANY'),'home_ownership']= 'OTHER'

Dikarenakan nilai NONE dan ANY hanya berjumlah sedikit dan cenderung memiliki makna yang hampir mirip dengan OTHER, oleh karena itu nilai tersebut digabungkan menjadi OTHER.

#### Dataset Unsupervised

Fitur Dataset Unsupervised
1. dti = Rasio dari hutang bulanan terhadap total utang dibagi dengan pendapatan bulanan                      
2. delinq_2yrs = Jumlah insiden tunggakan selama lebih dari 30 hari dalam catatan kredit peminjam selama 2 tahun terakhir.         
3. inq_last_6mths          
4. open_acc = Jumlah garis kredit yang masih aktif dalam catatan kredit peminjam.               
5. pub_rec = Jumlah catatan publik yang merugikan                 
6. revol_bal = Total saldo kredit yang dapat diputar              
7. initial_list_status = Status penulisan awal dari pinjaman. Nilai yang mungkin adalah - "Whole" (Utuh) atau "Fractional" (Fraksional).      
8. collection_recovery_fee = biaya penagihan setelah hutang di-charge off
9. acc_now_delinq = Jumlah akun di mana peminjam saat ini mengalami keterlambatan pembayaran.

In [25]:
df_unsupervised.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466281 entries, 1077501 to 9199665
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   dti                      466281 non-null  float64
 1   delinq_2yrs              466256 non-null  float64
 2   inq_last_6mths           466256 non-null  float64
 3   open_acc                 466256 non-null  float64
 4   pub_rec                  466256 non-null  float64
 5   revol_bal                466281 non-null  int64  
 6   initial_list_status      466281 non-null  object 
 7   collection_recovery_fee  466281 non-null  float64
 8   acc_now_delinq           466256 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 35.6+ MB


In [26]:
df_unsupervised[df_unsupervised['delinq_2yrs'].isna()]

,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,initial_list_status,collection_recovery_fee,acc_now_delinq
id,,,,,,,,,
71823,4.00,NaN,NaN,NaN,NaN,0,f,0.0000,NaN
72197,10.00,NaN,NaN,NaN,NaN,0,f,0.0000,NaN
73003,10.00,NaN,NaN,NaN,NaN,0,f,0.0000,NaN
72819,10.00,NaN,NaN,NaN,NaN,0,f,0.0000,NaN
91023,10.00,NaN,NaN,NaN,NaN,0,f,0.0000,NaN
85961,3.27,NaN,NaN,NaN,NaN,0,f,0.0000,NaN
85818,3.51,NaN,NaN,NaN,NaN,0,f,0.0000,NaN
85781,8.61,NaN,NaN,NaN,NaN,0,f,0.0000,NaN
85675,16.27,NaN,NaN,NaN,NaN,0,f,0.0000,NaN


In [27]:
drop_index_1 = df_unsupervised[df_unsupervised['delinq_2yrs'].isna()].index.values

In [28]:
#dilakukan drop berdasarkan index kepada 2 dataset supervised dan unsupervised
df_supervised = df_supervised.drop(drop_index_1)
df_unsupervised = df_unsupervised.drop(drop_index_1)